In [16]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

with open('intents.json') as f:
    intents = json.load(f)

## Data preprocessing

In [2]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [20]:
# get text and intent title from json data
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []

    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])

    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

intent_doc

{'Greeting': ['Hi human, please tell me your GeniSys user',
  'Hello human, please tell me your GeniSys user',
  'Hola human, please tell me your GeniSys user'],
 'GreetingResponse': ['Great! Hi <HUMAN>! How can I help?',
  'Good! Hi <HUMAN>, how can I help you?',
  'Cool! Hello <HUMAN>, what can I do for you?',
  'OK! Hola <HUMAN>, how can I help you?',
  'OK! hi <HUMAN>, what can I do for you?'],
 'CourtesyGreeting': ['Hello, I am great, how are you? Please tell me your GeniSys user',
  'Hello, how are you? I am great thanks! Please tell me your GeniSys user',
  'Hello, I am good thank you, how are you? Please tell me your GeniSys user',
  'Hi, I am great, how are you? Please tell me your GeniSys user',
  'Hi, how are you? I am great thanks! Please tell me your GeniSys user',
  'Hi, I am good thank you, how are you? Please tell me your GeniSys user',
  'Hi, good thank you, how are you? Please tell me your GeniSys user'],
 'CourtesyGreetingResponse': ['Great! Hi <HUMAN>! How can I hel

In [22]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')

    tokenizer.fit_on_texts(input_list)

    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')

    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [24]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])

    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

## Building the model

In [6]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [7]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         66048     
                                                                 
 bidirectional (Bidirectiona  (None, 256)              656384    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758,166
Trainable params: 758,166
Non-trainable params: 0
__________________________________________________

In [8]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 3s 29ms/step - loss: 3.0847 - accuracy: 0.0280
Epoch 2/50
5/5 [==============================] - 0s 29ms/step - loss: 3.0515 - accuracy: 0.0909
Epoch 3/50
5/5 [==============================] - 0s 29ms/step - loss: 3.0077 - accuracy: 0.1608
Epoch 4/50
5/5 [==============================] - 0s 29ms/step - loss: 2.9633 - accuracy: 0.2098
Epoch 5/50
5/5 [==============================] - 0s 28ms/step - loss: 2.8910 - accuracy: 0.2657
Epoch 6/50
5/5 [==============================] - 0s 29ms/step - loss: 2.7723 - accuracy: 0.3217
Epoch 7/50
5/5 [==============================] - 0s 29ms/step - loss: 2.6259 - accuracy: 0.3846
Epoch 8/50
5/5 [==============================] - 0s 31ms/step - loss: 2.3279 - accuracy: 0.3846
Epoch 9/50
5/5 [==============================] - 0s 30ms/step - loss: 2.0607 - accuracy: 0.4336
Epoch 10/50
5/5 [==============================] - 0s 31ms/step - loss: 1.8259 - accuracy: 0.4406
Epoch 11/50
5/5 [============

In [9]:
def predict_intent(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))

    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    class_pred = np.argmax(pred.numpy(), axis=1)[0]
    print(class_pred)
    return class_pred

def generate_response(intent):
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[intent]]), trg_index_word[intent]

## Chatbot

In [14]:
import gradio as gr
import time
import random

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        print(user_message)
        intent = predict_intent(user_message)
        print('intent:', intent)
        # Random choice randomly chooses one of the options that matches the intent

        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        # use_gpt = intent[0] == 'privacy_policy' or intent[0] == 'legal_statement'
        # response = gpt_model.answer_question(question=user_message) if use_gpt else 'No idea, bitch'
        response = generate_response(intent)

        # response = random.choice(response_map[intent])
        history[-1][1] = "{} (intent: {})".format(response[0], response[1])
        # The sleep is to simulate a more natural conversation
        time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()



Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


How are you?
intent: 2
Hello!
intent: 18
Hello
intent: 18
Hi
intent: 18
I need help
intent: 18
